In [9]:
import csv

counter = 0

# Open file 
with open('NonTravel.csv', encoding='utf-8') as file_obj:
      
    # Create reader object by passing the file 
    # object to reader method
    reader_obj = csv.reader(file_obj)
      
    # Iterate over each row in the csv 
    # file using reader object
    for row in reader_obj:
        # print(row)
        counter+=1
        with open("C:/Users/tl_ch/Desktop/Classification/Train/NonTravel/NonTravel_" + str(counter) + ".txt", "w", encoding='utf-8') as f:
            f.write(row[0])
            

['do', 'you', 'have', 'to', 'book', 'the', 'pcr', ...]


In [1]:
import nltk
from nltk.corpus import PlaintextCorpusReader

travel = PlaintextCorpusReader('Classification/Train/Travel', '.+\.txt')
nonTravel = PlaintextCorpusReader('Classification/Train/NonTravel', '.+\.txt')

travel_docs1 = [travel.words(fid) for fid in travel.fileids()]
nonTravel_docs1 = [nonTravel.words(fid) for fid in nonTravel.fileids()]

print(travel_docs1[0][0:20])
print(nonTravel_docs1[0][0:20])

['I', 'applaud', 'countries', 'who', 'show', 'no', 'favours', 'to', 'SG', 'vexsinated', 'travellers', 'because', 'they', 'aren', '&#', '39', ';', 't', 'discriminatory', ',']
['@', 'Ujong', 'Okoi', 'My', 'money', 'stays', 'right', 'in', 'my', 'tradıng', 'account', ',', 'my', '<', 'br', '>', 'account', 'just', 'mirrors', 'her']


In [2]:
# Combine the categories of the corpus
all_docs1 = travel_docs1 + nonTravel_docs1
num_travel_docs = len(travel_docs1)
print(num_travel_docs)
print (len(nonTravel_docs1))

150
150


In [3]:
# Processsing for stopwords, alphabetic words, Stemming 

all_docs2 = [[w.lower() for w in doc] for doc in all_docs1]

import re
all_docs3 = [[w for w in doc if re.search('^[a-z]+$',w)] for doc in all_docs2]

from nltk.corpus import stopwords
stop_list = stopwords.words('english')
all_docs4 = [[w for w in doc if w not in stop_list] for doc in all_docs3]

from nltk.stem.porter import *
stemmer = PorterStemmer()
all_docs5 = [[stemmer.stem(w) for w in doc] for doc in all_docs4]

In [4]:
#Create dictionary
import gensim
from gensim import corpora
dictionary = corpora.Dictionary(all_docs5)
print(dictionary)

Dictionary(1671 unique tokens: ['applaud', 'citizen', 'countri', 'discriminatori', 'favour']...)


C:\Users\tl_ch\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [5]:
# Convert all documents to TF Vectors
all_tf_vectors = [dictionary.doc2bow(doc) for doc in all_docs5]

In [6]:
all_data_as_dict = [{id:1 for (id, tf_value) in vec} for vec in all_tf_vectors]
print(type(all_data_as_dict))

travel_data = [(d, 'Travel') for d in all_data_as_dict[0:num_travel_docs]]
nonTravel_data = [(d, 'NonTravel') for d in all_data_as_dict[num_travel_docs:]]
all_labeled_data = travel_data + nonTravel_data

<class 'list'>


Naive Bayes Classifier

In [7]:
#Generate the trained classifier
classifier = nltk.NaiveBayesClassifier.train(all_labeled_data)

In [8]:
test_doc = all_data_as_dict[0]
print(classifier.classify(test_doc))

Travel


In [9]:
test_doc = all_data_as_dict[150] 
print(classifier.classify(test_doc))

NonTravel


Classification on new Data

In [11]:
# Read the files in validate folder and preparing the validation corpus

travel_validation = PlaintextCorpusReader('Classification/Validation/Travel', '.+\.txt')
nonTravel_validation = PlaintextCorpusReader('Classification/Validation/NonTravel', '.+\.txt')

# Tokenization
travel_valid_docs1 = [travel_validation.words(fid) for fid in travel_validation.fileids()]
nonTravel_valid_docs1 = [nonTravel_validation.words(fid) for fid in nonTravel_validation.fileids()]

# Combine the two sets of documents for easy processing.
all_valid_docs = travel_valid_docs1 + nonTravel_valid_docs1

# This number will be used to separate the two sets of documents later.
num_travel_valid_docs = len(travel_valid_docs1)

# Text pre-processing, including stop word removal, stemming, etc.
all_valid_docs2 = [[w.lower() for w in doc] for doc in all_valid_docs]
all_valid_docs3 = [[w for w in doc if re.search('^[a-z]+$',w)] for doc in all_valid_docs2]
all_valid_docs4 = [[w for w in doc if w not in stop_list] for doc in all_valid_docs3]
all_valid_docs5 = [[stemmer.stem(w) for w in doc] for doc in all_valid_docs4]

# Note that we're using the dictionary created earlier.
all_valid_tf_vectors = [dictionary.doc2bow(doc) for doc in all_valid_docs5]

# Convert documents into dict representation.
all_valid_data_as_dict = [{id:1 for (id, tf_value) in vec} for vec in all_valid_tf_vectors]

# Separate the two sets of documents and add labels.
travel_valid_data_with_labels = [(d, 'Travel') for d in all_valid_data_as_dict[0:num_travel_valid_docs]]
nonTravel_valid_data_with_labels = [(d, 'NonTravel') for d in all_valid_data_as_dict[num_travel_valid_docs:]]

# Combine the labeled documents.
all_valid_data_with_labels = travel_valid_data_with_labels + nonTravel_valid_data_with_labels

In [12]:
print(nltk.classify.accuracy(classifier, all_valid_data_with_labels))

0.573374613003096


In [155]:
# test_corpus = PlaintextCorpusReader('C:/Users/tl_ch/Desktop/Classification/Test', '.+\.txt')
test_corpus = PlaintextCorpusReader('Classification/Test', '.+\.txt')
fids = test_corpus.fileids()

# Tokenization
test_docs1 = [test_corpus.words(fid) for fid in fids]

# Text pre-processing, including stop word removal, stemming, etc.
test_docs2 = [[w.lower() for w in doc] for doc in test_docs1]
test_docs3 = [[w for w in doc if re.search('^[a-z]+$',w)] for doc in test_docs2]
test_docs4 = [[w for w in doc if w not in stop_list] for doc in test_docs3]
test_docs5 = [[stemmer.stem(w) for w in doc] for doc in test_docs4]

# Note that we're using the dictionary created earlier to create TF vectors
test_tf_vectors = [dictionary.doc2bow(doc) for doc in test_docs5]

# Convert documents into dict representation. This is document-label representation
test_data_as_dict = [{id:1 for (id, tf_value) in vec} for vec in test_tf_vectors]

import shutil
#For each file, classify and print the label.
for i in range(len(fids)):
    print(fids[i], '-->', classifier.classify(test_data_as_dict[i]))
    
    if (classifier.classify(test_data_as_dict[i]) == "Travel"):
        original = r'Classification/Test/'+fids[i]
        target = r'Classification/Output/'+fids[i]

        shutil.copyfile(original, target)

Test_10.txt --> Travel
Test_100.txt --> Travel
Test_1000.txt --> NonTravel
Test_10000.txt --> Travel
Test_10001.txt --> Travel
Test_10002.txt --> Travel
Test_10003.txt --> Travel
Test_10004.txt --> Travel
Test_10005.txt --> NonTravel
Test_10006.txt --> Travel
Test_10007.txt --> Travel
Test_10008.txt --> Travel
Test_10009.txt --> NonTravel
Test_1001.txt --> NonTravel
Test_10010.txt --> NonTravel
Test_10011.txt --> Travel
Test_10012.txt --> NonTravel
Test_10013.txt --> NonTravel
Test_10014.txt --> NonTravel
Test_10015.txt --> NonTravel
Test_10016.txt --> Travel
Test_10017.txt --> Travel
Test_10018.txt --> Travel
Test_10019.txt --> Travel
Test_1002.txt --> Travel
Test_10020.txt --> Travel
Test_10021.txt --> Travel
Test_10022.txt --> NonTravel
Test_10023.txt --> Travel
Test_10024.txt --> NonTravel
Test_10025.txt --> Travel
Test_10026.txt --> Travel
Test_10027.txt --> Travel
Test_10028.txt --> Travel
Test_10029.txt --> Travel
Test_1003.txt --> Travel
Test_10030.txt --> NonTravel
Test_10031.

In [152]:
classifier.show_most_informative_features(5)

Most Informative Features
                       9 = 1              Travel : NonTra =     12.3 : 1.0
                     678 = 1              NonTra : Travel =      9.0 : 1.0
                     676 = 1              NonTra : Travel =      9.0 : 1.0
                     355 = 1              Travel : NonTra =      7.7 : 1.0
                     569 = 1              NonTra : Travel =      7.4 : 1.0


In [13]:
import nltk
from nltk.corpus import PlaintextCorpusReader

travel = PlaintextCorpusReader('Classification/Output', '.+\.txt')
travel_docs = [travel.words(fid) for fid in travel.fileids()]

print(travel_docs[0])

['do', 'you', 'have', 'to', 'book', 'the', 'pcr', ...]


In [14]:
# Processsing for stopwords, alphabetic words, Stemming 

travel_docs2 = [[w.lower() for w in doc] for doc in travel_docs]

import re
travel_docs3 = [[w for w in doc if re.search('^[a-z]+$',w)] for doc in travel_docs2]

from nltk.corpus import stopwords
stop_list = stopwords.words('english')
travel_docs4 = [[w for w in doc if w not in stop_list] for doc in travel_docs3]

# from nltk.stem.porter import *
# stemmer = PorterStemmer()
# travel_docs5 = [[stemmer.stem(w) for w in doc] for doc in travel_docs4]

In [19]:
counter = 0
for c in travel_docs4:
    counter+=1
    output_file = open('Classification/Output_Cleaned/travel_'+str(counter)+'.txt', 'w')
    for w in c:
        output_file.write(w + '\n')
    output_file.close()

In [ ]:
output_file = open('months.txt', 'w')

for month in months:
    output_file.write(month)

output_file.close()